Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

### Инструкция по выполнению проекта

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

### Описание данных

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

# 1. Подготовка

In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.corpus import stopwords as nltk_stopwords
import nltk
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet
import re

data = pd.read_csv('toxic_comments.csv')

display(data.head())
print(data.info())

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB
None


Загрузили и проверили данные на пропуски на всякий случай, приступим к лемматизации

In [2]:
nltk.download('averaged_perceptron_tagger')

l = WordNetLemmatizer()

def clear_lemmatize(text):
    text = re.sub(r'[^a-zA-Z]', ' ', text).split()
    text = " ".join(text)
    word_list = nltk.word_tokenize(text)
    return ' '.join(l.lemmatize(w, wordnet.VERB) for w in word_list)

data['lemm_text'] = data['text'].apply(clear_lemmatize)

display(data.head())

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


,text,toxic,lemm_text
0,Explanation\nWhy the edits made under my usern...,0,Explanation Why the edit make under my usernam...
1,D'aww! He matches this background colour I'm s...,0,D aww He match this background colour I m seem...
2,"Hey man, I'm really not trying to edit war. It...",0,Hey man I m really not try to edit war It s ju...
3,"""\nMore\nI can't make any real suggestions on ...",0,More I can t make any real suggestions on impr...
4,"You, sir, are my hero. Any chance you remember...",0,You sir be my hero Any chance you remember wha...


Текст лемматизировали и очистили от лишних символов, в т.ч. с применением POS-TAG'а для того чтобы перевести слова в настоящее время (если я правильно понял как оно работает, но конечная метрика с пос-тэгом улучшилась). Теперь непосредственно можно перейти к разделению выборок на трейн-тест

In [3]:
train, test = train_test_split(data, test_size=0.20, random_state=666)

X_train = train['lemm_text']
y_train = train['toxic']

X_test = test['lemm_text']
y_test = test['toxic']

print(train.shape)
print(test.shape)

(127656, 3)
(31915, 3)


Выборки разделены на train и test, теперь перейдём непосредственно к TF-IDF

In [4]:
corpus = train['lemm_text'].values.astype('U')

nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

count_tf_idf = TfidfVectorizer(stop_words=stopwords)

train_final = count_tf_idf.fit_transform(corpus)

print(train_final.shape)

corpus_test = test['lemm_text'].values.astype('U')
test_final = count_tf_idf.transform(corpus_test)

print(test_final.shape)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


(127656, 141867)
(31915, 141867)


Создали корпусы и высчитали TF-IDF для обеих выборок (внимание, на практикумовском ноутбуке иногда падает, через раз как-то)

P.S. Если саму ячейку перезапустить ядро ноутбука почти всегда падает словно, это происходит на этапе перезаписи корпуса, если корпус удалить с помощью del и высчитать снова не падает

# 2. Обучение

In [5]:
model = LogisticRegression(class_weight='balanced')
model.fit(train_final, y_train)

answers = model.predict(test_final)

print('F1 лог. регрессии:', f1_score(y_test, answers))

F1 лог. регрессии: 0.7512181916621548


C:\Users\hp\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Итоговая метрика F1 на тесте показала 0.751 что ровно достаточно для прохождения порога для сдачи проекта. Я бы рад попробовать другие модели, но на деле большинство из них от работы с векторами текстов просто отлетают вместе с ядром юпитера, а те что не отлетают показывают результаты хуже.

# 3. Выводы

Стояла задача обучить модель предсказывающую является ли текст токсичным. Был вариант применить BERT (пробовал в т.ч. и distilBert), но к сожалению он у меня улетал на этапе Embedding'ов вместе с ядром юпитера, даже когда я взял батчи по 2 и сделал срез данных в виде первых 20 строк - ноутбук умер.

Поэтому было решено применить обычный TF-IDF, для этого текст был лемматизирован, очищен с помощью reg-exp от лишних символов, приведён к форме настоящего времени и после этого переведён в векторы с помощью TF-IDF.

Использовал логистическую регрессию как самый быстрый алгоритм, пробовал ещё SGD и KNN - первый выдал результаты значительно хуже лог. рега, от второго просто отвалился ноутбук, после чего было принято решение довольствоваться тем что есть т.к. задача уже выполнена и значение метрики F1 = 0.751

# Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны